# Find All Paid Parking Zones

Here, I find which blocks have paid parking zones. Doing an API request for Unique blockfaces just hangs (there are a lot of records in the database). Instead, I took a look at the data using the online portal, and found min and max values for blockface IDs. I then make a request for blockface info for each potential blockface ID. Each request only requires a small amount of data to transfer and as such is quite quick.

Requests are made using the socrata API with the city of seattle data (https://data.seattle.gov/Transportation/2019-Paid-Parking-Occupancy-Year-to-date-/qktt-2bsy). Data is output to a csv (block_locations.csv).

In [1]:
import pandas as pd
from datetime import timedelta, date
from sodapy import Socrata
import requests

In [2]:
# import app tokens, secret key for API calls
import os, sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
import app_tokens

In [24]:
#data is delayed 48 hrs
# socrata data keys for parking data
#2019 ytd
data_ytd = 'qktt-2bsy'
#last 30 days
data_mtd = 'rke9-rsvs'
# last 48 hours
data_48hrs = 'hiyf-7edq'


In [36]:
# unpack location object into latitude and longitude
def unpackCoordinates(location_obj):
    lat = location_obj['coordinates'][0]
    long = location_obj['coordinates'][1]
    # all seattle lat/long should be approx 47, -122. swap lat and long if wrong way
    if lat < long:
        return pd.Series([long, lat])
    else:
        return pd.Series([lat, long])

In [59]:
def getOneLocation(client, data_key, elementkey):

    results = client.get(data_key,
                     select='sourceelementkey,location',
                     sourceelementkey=elementkey,
                     limit=1)
    #print(results)
    return results

In [67]:
all_results = []

#initialize client
client = Socrata('data.seattle.gov',
                 app_tokens.getAppTokens()['seattle_gov'],
                timeout=60)
# loop through possible values
start = 1000
stop = 137000
for i in range(start, stop):
    print('i: %5d\tpercent done : %.3f' % (i, 100*(i - start) /(stop-start)), end='\r')
    results = getOneLocation(client=client, data_key=data_ytd, elementkey=i)
    #print(len(results))
    if len(results) > 0:
        all_results.append(results[0])

# close connection
client.close()

In [68]:
df_all = pd.DataFrame.from_records(all_results)
df_all['sourceelementkey'] = df_all['sourceelementkey'].apply(pd.to_numeric)
df_all = df_all.sort_values(by='sourceelementkey')
df_all[['latitude', 'longitude']] = df_all.location.apply(unpackCoordinates)

In [69]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1505 entries, 0 to 1504
Data columns (total 4 columns):
location            1505 non-null object
sourceelementkey    1505 non-null int64
latitude            1505 non-null float64
longitude           1505 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 58.8+ KB


In [70]:
df_all.to_csv('block_locations.csv')